In [2]:
from datasets import load_from_disk
import json

all_data = []
# ds_DBAR = load_from_disk("/home/lesong/codes/SimPO/datasets/llama3.1_8B_ultrafeedback/cpo_dataset_1")
with open("/home/lesong/codes/SimPO/on_policy_data_gen/datasets/llama3.1_8B_ultrafeedback/all_outputs_rm.json", "r") as f:
    ds_DBAR = json.load(f)

for item in ds_DBAR:
    all_data.append({
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"],
        "type": "DBAR"
    })

/home/lesong/anaconda3/envs/simpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import jsonlines
all_data = []

with jsonlines.open("/home/lesong/codes/SimPO/data/cpo_data.json") as reader:
    for item in reader:
        all_data.append({
            "prompt": item["prompt"],
            "chosen": item["chosen"],
            "rejected": item["rejected"],
            "type": "D"
    })

with jsonlines.open("/home/lesong/codes/SimPO/data/all_train_data.json") as reader:
    for item in reader:
        if item["prompt"].startswith("Your task is to decide whether question decomposition is necessary to answer a given question"): continue
        all_data.append({
        "prompt": item["prompt"],
        "chosen": item["chosen"],
        "rejected": item["rejected"],
        "type": "DBAR"
    })

        
with jsonlines.open("data/all_train_data.json", "w") as writter:
    writter.write_all(all_data)

In [2]:
import json
import jsonlines
from vllm import LLM, SamplingParams
from datasets import load_dataset, load_from_disk
import os
import json





llm = LLM(model=args.model)
tokenizer = llm.get_tokenizer()

train_dataset= load_dataset(data_dir, split='train_prefs')

prompts = sorted(list(set(train_dataset['prompt'])))

conversations = [tokenizer.apply_chat_template([{'role': 'user', 'content': prompt}], tokenize=False, add_generation_prompt=True) for prompt in prompts]

sampling_params = SamplingParams(temperature=args.temperature, 
                                 top_p=args.top_p, 
                                 max_tokens=args.max_tokens, 
                                 seed=args.seed,)
outputs = llm.generate(conversations, sampling_params)

# Save the outputs as a JSON file.
output_data = []
for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    output_data.append({
        'prompt': prompts[i],
        "format_prompt": prompt,
        'generated_text': generated_text,
    })


all_data = []
with jsonlines.open("./on_policy_data_gen/datasets/llama3.1_8B_ultrafeedback/cpo_data.json") as reader:
    for obj in reader:
        obj["type"] = "D"
        obj["rejected"][1]["content"] = 
        print(obj)
        break

{'prompt': 'Your task is to decide whether question decomposition is necessary to answer a given question, if yes, which sub-question shall be asked. \n    Do not ask same or similar sub-questions that you have asked before.\n    Decompose the question in a reasonable and smart way. \n    Only give one sub-question each time.\n    Sub questions that you have asked before, if any:\n                \n                \n    Think step by step.\n    Based on the given context, provide your answer to the following question: What is the river where Long Spruce Generating Station is located a tributary of?\n    Your answer shall be in the following JSON format:\n    {\n        "decomposition": true, \n        "sub question": <A sub question that you would like to ask>\n    }\n    OR\n    {"decomposition": false}\n    ', 'chosen': [{'content': 'Your task is to decide whether question decomposition is necessary to answer a given question, if yes, which sub-question shall be asked. \n    Do not a

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "/home/lesong/codes/SimPO/outputs/llama-3-3b-instruct-simpo-v2_1"

model_name = "Qwen/Qwen2.5-3B-Instruct"

# model_name = "meta-llama/Llama-3.2-3B-Instruct"

# llm_model = AutoModelForCausalLM.from_pretrained(model_id)
# model.load_adapter(peft_model_id)

llm_model = AutoModelForCausalLM.from_pretrained(
    # "meta-llama/Llama-3.2-3B-Instruct",
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
# llm_model.load_adapter(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def parse_decompose_response(raw_resp):
    resp = {"decomposition": False}
    start_idx, end_idx = raw_resp.find("<sub question>"), raw_resp.find("</sub question>")
    if start_idx >= 0 and end_idx >= 0:
        sub_question = raw_resp[start_idx+len("<sub question>"):end_idx].strip()
        resp["sub question"] = sub_question
        start_idx, end_idx = raw_resp.find("<decomposition>"), raw_resp.find("</decomposition>")
        if start_idx >= 0 and end_idx >= 0:
            decomposition = (True if raw_resp[start_idx+len("<decomposition>"):end_idx].strip().lower() == "true" else False)
            resp["decomposition"] = decomposition
    return resp



/home/lesong/anaconda3/envs/simpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


In [2]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("/home/lesong/codes/SimPO/datasets/llama3.1_8B_ultrafeedback/cpo_dataset_1")
dataset['test']

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'type'],
    num_rows: 1482
})

In [ ]:
import numpy as np

prompt = np.random.choice(dataset["test"])


def get_response(prompt, llm_model, tokenizer):
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt["prompt"]}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(llm_model.device)
    
    terminators = [
            tokenizer.eos_token_id,
            # tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    

    generated_ids = llm_model.generate(
        **model_inputs,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id = tokenizer.eos_token_id,
        temperature=1.2, 
        top_p=1.2, 
        top_k=20, 
        num_beams=5,
        do_sample=True
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print("responses: ", responses)
    sub_questions = []
    for resp in responses:
        resp = parse_decompose_response(resp)
        if resp["decomposition"]:
            sub_questions.append(resp["sub question"])
    
    return list(set(sub_questions))

for i in range(5):
    print("round: ", i)
    print(get_response(prompt, llm_model, tokenizer))
    print(prompt["chosen"][1]["content"])

round:  0
responses:  ['<decomposition>true</decomposition>\n<sub_question>Who was the king of England in 1951?</sub_question>']
[]
<decomposition>true</decomposition> 
    <sub question>What is the nationality of Lionel Logue, the speech and language therapist in "The King's Speech"?</sub question>
round:  1
responses:  ['<decomposition>true</decomposition>\n<sub_question>Who was the king of England in 1951?</sub_question>']
[]
<decomposition>true</decomposition> 
    <sub question>What is the nationality of Lionel Logue, the speech and language therapist in "The King's Speech"?</sub question>
round:  2
responses:  ['<decomposition>true</decomposition>\n<sub_question>Who was the king of England in 1951?</sub_question>']
[]
<decomposition>true</decomposition> 
    <sub question>What is the nationality of Lionel Logue, the speech and language therapist in "The King's Speech"?</sub question>
round:  3
responses:  ['<decomposition>true</decomposition>\n<sub_question>Who was the king of En

: 